# Segundo Examen Parcial
# Computacion Concurrente
### Martiñón Luna Jonathan José
### Fecha: 31 - Octubre - 2019

---


# Programa 1  


<div style="text-align: justify">
Implementa una simulación de la fábula que cuenta la carrera entre la liebre y la tortuga. Para hacerlo más interesante la carrera será cuesta arriba por una pista resbaladiza, de modo que a veces podrán resbalar y retroceder algunas posiciones. Habrá un $Thread$ (instanciado con clases) que implementará a la tortuga y otro la liebre. Cada uno se suspenderá durante un segundo y luego evaluará lo que ha pasado según unas probabilidades:
</div>

* Tortuga:
    * 50% Avance rápido **3 casillas arriba**
    * 20% Resbaló **6 casillas abajo**
    * 30% Avance lento **1 casilla arriba**
* Liebre:
    * 20% Duerme **No hay avance**
    * 20% Gran salto **9 casillas arriba**
    * 10% Resbalón grande **12 casillas abajo**
    * 30% Pequeño salto **1 casilla arriba**
    * 20% Resbalón pequeño **2 casillas abajo**

<div style="text-align: justify">
Calcula la probabilidad con random, de 1 a 100 y determina con dicho número qué ha hecho cada animal. Considera que hay 70 casillas, de la 1 a la 70, la 1 de salida y la 70 de llegada. Si resbala al principio vuelve a la 1, nunca por debajo. Tras cada segundo y después de de calcular su nueva posición escribe una línea por cada animal, con espacios en blanco de la 1 la posición-1 y luego una letra T para la tortuga y una L para la liebre. Imprima al comienzo de la carrera un mensaje. Después de imprimir las líneas determine si alguno ha llegado a la meta y ha ganado, escribiendo un mensaje. Si ambos llegan a la vez declare empate.
</div>

In [1]:
#Libreria para nuestros hilos
import threading
#Libreria para nuestras colas
import queue
#Libreria para los numeros aleatorios
import random
#Para nuestros tiempos
import time

Creando nuestra clase mutex (Bloqueara y desbloqueará)

In [2]:
# Creando nuestra clase mutex
class Mutex:
    
    #Iniciamos creando nuestro seguro
    def __init__(self):
        self.lock = threading.Lock()
        
    #Definimos la función encargada de realizar los bloqueos
    def bloquea(self):
        self.lock.acquire()
    
    #Definimos la función encargada de liberar los bloqueos
    def libera(self):
        self.lock.release()

Creando nuestra clase Tortuga

In [3]:
class Tortuga(threading.Thread):
    
    #Constructor inicial de la clase
    def __init__(self, mutex):
        #Llamamos al constructor del hilo (Thread)
        super().__init__()
        self.Mutex = mutex
        
    #Función que determinará la acción de nuestra tortuga
    def Accion(self):
        
        global Posicion_Tortuga
        global Carrera_Activa
        global meta
        
        accion = random.randint(1,100)
        
        #Aqui definimos nuestro 20%
        if accion> 0 and accion <21:
            
            print("Oh! no! Resbaló!!!")
            
            #Verificamos que nunca llegue a 0
            for i in range(1,7):
                if Posicion_Tortuga -1 == 0:
                    print("Regresó al inicio")
                    break
                else:
                    Posicion_Tortuga-=1            
            
        #Aquí definimos nuestro 30%
        elif accion > 20 and accion < 51:
            print("Avance lento")
            Posicion_Tortuga +=1 
            
            #En caso de que con nuestro incremento
            #Llegue a la meta, se terminará la carrera
            if Posicion_Tortuga == meta:
                Carrera_Activa = False
        
        #Como hablamos de un ambiente controlado
        #No es necesario especificar el otro intervao
        #Simplemente sabemos que de no estar en los
        #anteriores, estará en este caso.
        else:
            
            print("Avance rápido")
            
            #Verificaremos paso por paso
            #Si ya ha llegado a la meta
            #Para que no se pase
            for i in range(1,4):
                Posicion_Tortuga +=1
                if Posicion_Tortuga == meta:
                    Carrera_Activa = False
                    break
                    
        return Posicion_Tortuga
            
    def run(self):
        
        global Carrera_Activa,Turno1,Turno2, Posicion_Tortuga
        
        while Carrera_Activa:
            
            if Turno1:
                self.Mutex.bloquea()

                print("\n\nTortuga:")
                Posicion_Tortuga = self.Accion()
                #Dejamos pasar el segundo despues de la accion
                time.sleep(1)
                print("Se encuentra en la casilla",Posicion_Tortuga)
                #Imprimiedo el espacio de 1 a posicion -1
                print(" "*(Posicion_Tortuga-1),"T")
                if Carrera_Activa:
                    Turno2 = True
                    
                else:
                    Turno2 = False
                Turno1 = False
                

                self.Mutex.libera()
            
        
    

Creando nuestra clase liebre

In [4]:
class Liebre(threading.Thread):
    
    #Constructor inicial de la clase
    def __init__(self, mutex):
        #Llamamos al constructor del hilo (Thread)
        super().__init__()
        self.Mutex = mutex
        
    def Accion(self):
        
        global Posicion_Liebre
        global Carrera_Activa
        global meta
        accion = random.randint(1,100)
        
        #Definiendo nuestro 20%
        if accion > 0 and accion <21:
          
            print("Se quedó dormida!")
            Posicion_Liebre+=0
        
        #Definiendo nuestro otro 20%
        elif accion > 20 and accion < 41:
            
            print("Dió un gran salto!")
            
            for i in range(1,10):
                
                if Posicion_Liebre == meta:
                    Carrera_Activa = False
                    break
                else:
                    Posicion_Liebre +=1
        
        #Definiendo nuestro 10%
        elif accion > 40 and accion < 51:
            
            print("Tuvo un gran Resbalón!")
            
            for i in range(1,13):
                if Posicion_Liebre -1 == 0:
                    print("Regresó al inicio")
                    break
                else:
                    Posicion_Liebre-= 1
            
        #Definiendo nuestro 30%
        elif accion >50 and accion < 81:
            
            print("Dio un pequeño salto")
            Posicion_Liebre +=1
            
            if Posicion_Liebre == meta:
                Carrera_Activa = False
            
        #Nuevamente es un ambiente controlado, por lo
        #que lo demás es nuestro restante 20%
        else:
            
            print("Tuvo un pequeño resbalón")
            for i in range(1,3):
                if Posicion_Liebre -1 == 0:
                    print("Regresó al inicio")
                    break
                else:
                    Posicion_Liebre -= 1
                    
        return Posicion_Liebre
            
    def run(self):
        global Carrera_Activa,Turno1, Turno2, Posicion_Liebre
        
        while Carrera_Activa:
            
            if Turno2:
                
                self.Mutex.bloquea()
                print("\n\nLiebre:")
                Turno1 = True
                Posicion_Liebre = self.Accion()
                time.sleep(1)
                print("Se encuentra en la casilla",Posicion_Liebre)
                print(" "*(Posicion_Liebre-1),"L")
                if Carrera_Activa:
                    Turno1 = True
                    
                else:
                    Turno1 = False
                Turno2 = False
                self.Mutex.libera()

Ejecutor (main)

In [5]:
t = time.time()  
meta = 70
Posicion_Tortuga = 1
Posicion_Liebre = 1
Carrera_Activa= True
Turno1 = True
Turno2 = True

mutex = Mutex()

print("\n\t\t...COMIENZA LA CARRERA!!!")

#Anteriormente había  puesto un 
#while Carrera_Activa
#Sin embargo parecía no detectarlo
while Carrera_Activa:
    
    tortuga = Tortuga( mutex)
    liebre = Liebre( mutex)

    tortuga.start()
    liebre.start()

    tortuga.join()
    liebre.join()

#Si terminó la carrera No debería haber aumento de posicion    
if Posicion_Tortuga==Posicion_Liebre:
    print("\n\n\t...HAN EMPATADO!!")
elif Posicion_Tortuga > Posicion_Liebre:
    print("\n\n\t...GANÓ LA TORTUGA!!")
else:
    print("\n\n\t...GANÓ LA LIEBRE!!")

    
print("\n\nTiempo de ejecución: ", time.time()-t)


		...COMIENZA LA CARRERA!!!


Tortuga:
Oh! no! Resbaló!!!
Regresó al inicio
Se encuentra en la casilla 1
 T


Liebre:
Se quedó dormida!
Se encuentra en la casilla 1
 L


Tortuga:
Avance rápido
Se encuentra en la casilla 4
    T


Liebre:
Dio un pequeño salto
Se encuentra en la casilla 2
  L


Tortuga:
Oh! no! Resbaló!!!
Regresó al inicio
Se encuentra en la casilla 1
 T


Liebre:
Dió un gran salto!
Se encuentra en la casilla 11
           L


Tortuga:
Oh! no! Resbaló!!!
Regresó al inicio
Se encuentra en la casilla 1
 T


Liebre:
Se quedó dormida!
Se encuentra en la casilla 11
           L


Tortuga:
Oh! no! Resbaló!!!
Regresó al inicio
Se encuentra en la casilla 1
 T


Liebre:
Tuvo un pequeño resbalón
Se encuentra en la casilla 9
         L


Tortuga:
Avance rápido
Se encuentra en la casilla 4
    T


Liebre:
Tuvo un gran Resbalón!
Regresó al inicio
Se encuentra en la casilla 1
 L


Tortuga:
Avance rápido
Se encuentra en la casilla 7
       T


Liebre:
Dio un pequeño salto
Se encuentra



Liebre:
Tuvo un pequeño resbalón
Se encuentra en la casilla 44
                                            L


Tortuga:
Avance rápido
Se encuentra en la casilla 28
                            T


Liebre:
Dió un gran salto!
Se encuentra en la casilla 53
                                                     L


Tortuga:
Avance rápido
Se encuentra en la casilla 31
                               T


Liebre:
Dió un gran salto!
Se encuentra en la casilla 62
                                                              L


Tortuga:
Avance rápido
Se encuentra en la casilla 34
                                  T


Liebre:
Dio un pequeño salto
Se encuentra en la casilla 63
                                                               L


Tortuga:
Avance rápido
Se encuentra en la casilla 37
                                     T


Liebre:
Dio un pequeño salto
Se encuentra en la casilla 64
                                                                L


Tortuga:
Avance rápido
Se encuentra en 

# Programa 2

<div style="text-align: justify">
Implementa un programa secuencial que calcule el producto de dos grandes matrices (de manera convencional, es decir, renglón por columna). Después modifícalo para que esta tarea se realice entre 4 $Threads$, cada uno ocupado de un subconjunto de la matriz resultado. Mide el tiempo que emplea cada una de las versiones. 
</div>


---

### Secuencialmente

In [1]:
import numpy as np
import random as rd
import time

In [2]:
tamanio= 100
#Se usaron 100 datos, pues si se usaban más, se tardaba muchísimo, lo que no me permitia ver si estaba o no bien#Capturando el tiempo a partir de que iniciamos
t = time.time()

#Creando las matrices
M1 = np.zeros((tamanio,tamanio))
M2 = np.zeros((tamanio,tamanio))

#Llenando los datos de la matriz
for i in range(tamanio):
    for j in range(tamanio):
        
        M1[i][j] = rd.randint(1,100)
        M2[i][j] = rd.randint(1,100)
        
#Creando nuestra matriz resultado de la multiplicacion
New_M = np.zeros((tamanio,tamanio))

for Ren in range(tamanio):
    #Tomando la primera columna completa
    aux = M1[Ren][:]
    for col in range (tamanio):
        #Tomando el primer renglon completo
        aux2 = M2[:][col]
        
        sum=0
        
        for k in range(tamanio):
            #Multiplicando renglon por columna 
            sum += aux[k]*aux2[k]
        
        #Agregando el valor en la matriz 
        New_M[Ren][col] = sum
        
       
print("A:\n",M1)
print("\nB:\n",M2)
print("\nA x B:\n",New_M)
print("\nTiempo de ejecución: ", time.time()-t)

A:
 [[92. 66. 61. ... 37. 80. 53.]
 [82. 20. 12. ... 32. 56. 60.]
 [40. 46.  1. ... 58. 32. 15.]
 ...
 [97. 90. 70. ... 54. 97. 67.]
 [82. 37. 11. ... 50. 67. 76.]
 [84. 75. 30. ... 43. 52. 21.]]

B:
 [[ 66.   9.  99. ...  82.  33.  39.]
 [ 28. 100.  60. ...  16.  76.  95.]
 [ 46.  48.  24. ...  65.   1.  40.]
 ...
 [ 73.  58.  66. ...  20.  51.  89.]
 [  8.  87.   8. ...  19.  94.   1.]
 [ 39.  23.  16. ...  75.  23.  87.]]

A x B:
 [[287981. 281539. 257258. ... 273469. 305098. 264421.]
 [245384. 243089. 239471. ... 225328. 250469. 229729.]
 [243132. 227996. 237033. ... 230242. 233649. 215956.]
 ...
 [285705. 292485. 266669. ... 296068. 312008. 283317.]
 [238317. 230268. 232788. ... 233735. 251443. 227938.]
 [258480. 269619. 262662. ... 251775. 275029. 258927.]]

Tiempo de ejecución:  0.5029139518737793


### Con hilos

In [1]:
#Libreria para nuestros hilos
import threading
#Libreria para nuestras colas
import queue
#Libreria para los numeros aleatorios
import random as rd
#Para nuestros tiempos
import time
#Para las matrices
import numpy as np

Clase mutex

In [2]:
# Creando nuestra clase mutex
class Mutex:
    
    #Iniciamos creando nuestro seguro
    def __init__(self):
        self.lock = threading.Lock()
        
    #Definimos la función encargada de realizar los bloqueos
    def bloquea(self):
        self.lock.acquire()
    
    #Definimos la función encargada de liberar los bloqueos
    def libera(self):
        self.lock.release()

In [3]:
class Multiplica1(threading.Thread):
    
    def __init__(self,name,mutex):
        super().__init__()
        self.name = name
        self.mutex = mutex
        
    def inicia(self,Ren):
        
        global M1,M2,New_Mat,tamanio
        
        aux = M1[Ren][:]
        for i in range(tamanio):
            aux2 = M2[:][i]
            suma=0
            for k in range(tamanio):
                #Multiplicando renglon por columna 
                suma += aux[k]*aux2[k]
            New_Mat[Ren][i] = suma
            #print("Calcule ",New_Mat[Ren][i])
            
    def run(self):
        
        global Calculando,Turno1,Turno2,Turno3,Turno4,cont
        
        while Calculando:
            if Turno1:
                self.mutex.bloquea()
                if cont < tamanio:

                    
                    print("Soy el hilo",self.name,"Trabajare con el renglon",cont)
                    self.inicia(cont)
                    print("Calcule ")
                    if Calculando:
                        Turno2 = True
                    else:
                        Turno2 = False
                    Turno1 = False
                    print("\n\n")
                    cont+=1
                    
                else:
                    Calculando = False
                self.mutex.libera()
    

In [4]:
class Multiplica2(threading.Thread):
    
    def __init__(self,name,mutex):
        super().__init__()
        self.name = name
        self.mutex = mutex
        
    def inicia(self,Ren):
        
        global M1,M2,New_Mat,tamanio
        
        aux = M1[Ren][:]
        for i in range(tamanio):
            aux2 = M2[:][i]
            suma=0
            for k in range(tamanio):
                #Multiplicando renglon por columna 
                suma += aux[k]*aux2[k]
            New_Mat[Ren][i] = suma
            #print("Calcule ",New_Mat[Ren][i])
            
    def run(self):
        
        global Calculando,Turno1,Turno2,Turno3,Turno4,cont,tamanio
        
        while Calculando:
            if Turno2:
                self.mutex.bloquea()
                if cont < tamanio:
                    print("Soy el hilo",self.name,"Trabajare con el renglon",cont)
                    self.inicia(cont)
                    print("Calcule ")
                    print("\n\n")
                    cont+=1
                    if Calculando:
                        Turno3 = True
                    else:
                        Turno3 = False
                    Turno2 = False
                else:
                    Calculando = False

                self.mutex.libera()

In [5]:
class Multiplica3(threading.Thread):
    
    def __init__(self,name,mutex):
        super().__init__()
        self.name = name
        self.mutex = mutex
    def inicia(self,Ren):
        
        global M1,M2,New_Mat,tamanio
        
        aux = M1[Ren][:]
        for i in range(tamanio):
            aux2 = M2[:][i]
            suma=0
            for k in range(tamanio):
                #Multiplicando renglon por columna 
                suma += aux[k]*aux2[k]
            New_Mat[Ren][i] = suma
            #print("Calcule ",New_Mat[Ren][i])
            
    def run(self):
        
        global Calculando,Turno1,Turno2,Turno3,Turno4,cont,tamanio
        
        while Calculando:
            if Turno3:
                self.mutex.bloquea()
                if cont < tamanio:
                    print("Soy el hilo",self.name,"Trabajare con el renglon",cont)
                    self.inicia(cont)
                    print("Calcule ")
                    cont+=1
                    if Calculando:
                        Turno4 = True
                    else:
                        Turno4 = False
                    Turno3 = False

                    print("\n\n")
                else:
                    Calculando = False
                self.mutex.libera()

In [9]:
class Multiplica4(threading.Thread):
    
    def __init__(self,name,mutex):
        super().__init__()
        self.name = name
        self.mutex = mutex
        
    def inicia(self,Ren):
        
        global M1,M2,New_Mat,tamanio
        
        aux = M1[Ren][:]
        for i in range(tamanio):
            aux2 = M2[:][i]
            suma=0
            for k in range(tamanio):
                #Multiplicando renglon por columna 
                suma += aux[k]*aux2[k]
            New_Mat[Ren][i] = suma
            #print("Calcule ",New_Mat[Ren][i])
            
    def run(self):
        
        global Calculando,Turno1,Turno2,Turno3,Turno4,cont,tamanio
        
        while Calculando:
            if Turno4:
                self.mutex.bloquea()
                
                if cont < tamanio:
                    print("Soy el hilo",self.name,"Trabajare con el renglon",cont)
                    self.inicia(cont)
                    print("Calcule ")
                    print("\n\n")
                    cont+=1
                    if(Calculando):
                        Turno1=True
                    else:
                        Turno1 = False
                    Turno4 = False
                else:
                    Calculando = False
                self.mutex.libera()

In [11]:
tamanio= 100
#Se usaron 100 datos, pues si se usaban más, se tardaba muchísimo, lo que no me permitia ver si estaba o no bien#Capturando el tiempo a partir de que iniciamos
t = time.time()

#Creando las matrices
M1 = np.zeros((tamanio,tamanio))
M2 = np.zeros((tamanio,tamanio))

Turno1 = True
Turno2 = False
Turno3 = False
Turno4 = False

#Llenando los datos de la matriz
for i in range(tamanio):
    for j in range(tamanio):
        
        M1[i][j] = rd.randint(1,100)
        M2[i][j] = rd.randint(1,100)
        
New_Mat = np.zeros((tamanio,tamanio))
mutex = Mutex()
Calculando = True

cont = 0

while Calculando:
    
    hilo1 = Multiplica1(1,mutex)
    hilo2 = Multiplica2(2,mutex)
    hilo3 = Multiplica3(3,mutex)
    hilo4 = Multiplica4(4,mutex)
    
    hilo1.start()
    hilo2.start()
    hilo3.start()
    hilo4.start()
    
    hilo1.join()
    hilo2.join()
    hilo3.join()
    hilo4.join()

print("La matriz A:\n",M1)
print("La matriz B:\n",M2)
print("La matriz final es:\n",New_Mat)
print("\nTiempo de ejecución: ", time.time()-t)

Soy el hilo 1 Trabajare con el renglon 0
Calcule 



Soy el hilo 2 Trabajare con el renglon 1
Calcule 



Soy el hilo 3 Trabajare con el renglon 2
Calcule 



Soy el hilo 4 Trabajare con el renglon 3
Calcule 



Soy el hilo 1 Trabajare con el renglon 4
Calcule 



Soy el hilo 2 Trabajare con el renglon 5
Calcule 



Soy el hilo 3 Trabajare con el renglon 6
Calcule 



Soy el hilo 4 Trabajare con el renglon 7
Calcule 



Soy el hilo 1 Trabajare con el renglon 8
Calcule 



Soy el hilo 2 Trabajare con el renglon 9
Calcule 



Soy el hilo 3 Trabajare con el renglon 10
Calcule 



Soy el hilo 4 Trabajare con el renglon 11
Calcule 



Soy el hilo 1 Trabajare con el renglon 12
Calcule 



Soy el hilo 2 Trabajare con el renglon 13
Calcule 



Soy el hilo 3 Trabajare con el renglon 14
Calcule 



Soy el hilo 4 Trabajare con el renglon 15
Calcule 



Soy el hilo 1 Trabajare con el renglon 16
Calcule 



Soy el hilo 2 Trabajare con el renglon 17
Calcule 



Soy el hilo 3 Trabajare con el renglon